Ce fichier est un version modifié du fichier du cours, il existe pour enregistrer en pickle les données qui me permettent de generer les patchs dans le script 5.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/University/ProjetML/')

In [ ]:
!ls 'Data/Donnees_ENT/Images'

20160322_S2A.tif  20160730_S2A.tif  20161018_S2A.tif  20161217_S2A.tif
20160710_S2A.tif  20160928_S2A.tif  20161127_S2A.tif  20161227_S2A.tif


## **Importation de quelques librairies**

In [ ]:
!pip install earthpy
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import rescale_intensity
import glob
import pickle
import pandas as pd
from matplotlib.colors import ListedColormap, BoundaryNorm
import random
import earthpy.plot as ep

     |████████████████████████████████| 1.4MB 8.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 18.3MB 171kB/s 
     |████████████████████████████████| 972kB 43.9MB/s 
     |████████████████████████████████| 14.8MB 232kB/s 
     |████████████████████████████████| 6.5MB 50.6MB/s 
  Created wheel for earthpy: filename=earthpy-0.9.2-cp36-none-any.whl size=1365088 sha256=c86290c12c3596a01c198eb22a13705a582e20f40ad0199ec3ab1e0b3a595ed5
  Stored in directory: /root/.cache/pip/wheels/27/f9/a5/026bb95ef7b12f9343b842da6897b4001c5b902550095cbbbe
Successfully built earthpy


## **Exploration de la vérité terrain**

In [ ]:
# Classes d'occupation du sol
gt_class = tifffile.imread ('Data/Donnees_ENT/Verite_terrain/DORDOGNE_VT_CLASS.tif')
gt_class.shape , gt_class.dtype

((5412, 5592), dtype('uint8'))

In [ ]:
# Identifiants
gt_id = tifffile.imread ('Data/Donnees_ENT/Verite_terrain/DORDOGNE_VT_ID.tif')
gt_id.shape, gt_id.dtype

((5412, 5592), dtype('int16'))

In [ ]:
labels = ["Zones urbaines", "Eau", "Forêt", "Landes","Cultures"]
cm = ListedColormap(['#909090','#0cd3ff','#09561f','#a3d153','#c20453'])

gt = gt_class.astype(float)
gt[np.where(gt_class==0)]=np.nan

# **Normalisation des données**

Dans le domaine du traitement des images satellitaires et notamment lorsqu'on travaille avec des séries temporelles, il est préférable de **normaliser les données par bande sur toute la série d'images** et non par date/image de sorte à ne pas bousculer la dynamique de certaines classes d'occupation du sol comme la végétation qui ont un profil temporel bien spécifique (ex. une courbe en cloche décrivant les changements dans le temps).

In [ ]:
# Récupérer la liste des images
lst_img = glob.glob ('Data/Donnees_ENT/Images/*.tif')
lst_img.sort() # ordonner par date si ce n'est pas le cas
lst_img

['Data/Donnees_ENT/Images/20160322_S2A.tif',
 'Data/Donnees_ENT/Images/20160710_S2A.tif',
 'Data/Donnees_ENT/Images/20160730_S2A.tif',
 'Data/Donnees_ENT/Images/20160928_S2A.tif',
 'Data/Donnees_ENT/Images/20161018_S2A.tif',
 'Data/Donnees_ENT/Images/20161127_S2A.tif',
 'Data/Donnees_ENT/Images/20161217_S2A.tif',
 'Data/Donnees_ENT/Images/20161227_S2A.tif']

In [ ]:
# Lecture de la bande du rouge (B1) pour toute la série temporelle
red_ts = []
for img in lst_img:
  red_ts.append( tifffile.imread(img)[:,:,0]) # Rouge
red_ts = np.dstack(red_ts)
red_ts.shape

(5412, 5592, 8)

In [ ]:
# Lecture de la bande du vert (B2) pour toute la série temporelle
green_ts = []
for img in lst_img:
  green_ts.append( tifffile.imread(img)[:,:,1]) # Vert
green_ts = np.dstack(green_ts)
green_ts.shape

(5412, 5592, 8)

In [ ]:
# Lecture de la bande du bleu (B3) pour toute la série temporelle
blue_ts = []
for img in lst_img:
  blue_ts.append( tifffile.imread(img)[:,:,2]) # Bleu
blue_ts = np.dstack(blue_ts)
blue_ts.shape

(5412, 5592, 8)

In [ ]:
# Lecture de la bande du proche infrarouge (B4) pour toute la série temporelle
nir_ts = []
for img in lst_img:
  nir_ts.append( tifffile.imread(img)[:,:,3]) # Proche infra rouge
nir_ts = np.dstack(nir_ts)
nir_ts.shape

(5412, 5592, 8)

**Normalisation Min Max entre 0 et 1**

Vous pouvez essayer une normalisation avec les percentiles 2 et 98 afin de tenir compte d'eventuelles valeurs aberrantes

In [ ]:
# Normalisation des séries temporelles par bande: Rouge
red_ts_norm = ( red_ts - red_ts.min() ) / ( red_ts.max() - red_ts.min() ).astype(np.float32)
red_ts_norm.min() , red_ts_norm.max(), red_ts_norm.shape, red_ts_norm.dtype

(0.0, 1.0, (5412, 5592, 8), dtype('float32'))

In [ ]:
# Normalisation des séries temporelles par bande: Vert
green_ts_norm = ( green_ts - green_ts.min() ) / ( green_ts.max() - green_ts.min() ).astype(np.float32)
green_ts = None
green_ts_norm.min() , green_ts_norm.max(), green_ts_norm.shape, green_ts_norm.dtype

(0.0, 1.0, (5412, 5592, 8), dtype('float32'))

In [ ]:
# Normalisation des séries temporelles par bande: Bleu
blue_ts_norm = ( blue_ts - blue_ts.min() ) / ( blue_ts.max() - blue_ts.min() ).astype(np.float32)
blue_ts = None
blue_ts_norm.min() , blue_ts_norm.max(), blue_ts_norm.shape, blue_ts_norm.dtype

(0.0, 1.0, (5412, 5592, 8), dtype('float32'))

In [ ]:
# Normalisation des séries temporelles par bande: Proche infrarouge
nir_ts_norm = ( nir_ts - nir_ts.min() ) / ( nir_ts.max() - nir_ts.min() ).astype(np.float32)
nir_ts_norm.min() , nir_ts_norm.max(), nir_ts_norm.shape, nir_ts_norm.dtype

(0.0, 1.0, (5412, 5592, 8), dtype('float32'))

# **Echantillons d'entraînement et Echantillons test**

In [ ]:
# Récupérer les positions des échantillons d'entraînement et test
idx_train_ = np.where ( (gt_id!=0) & (gt_class!=0) )
idx_test = np.where ( (gt_id!=0) & (gt_class==0) )

In [ ]:
# Lecture des données d'entraînement
train_id_ = gt_id[idx_train_]
train_y_ = gt_class[idx_train_]
f'échantillons d\'entrainement: {train_y_.shape[0]} pixels, {len(np.unique(train_id_))} objets'

"échantillons d'entrainement: 605431 pixels, 1859 objets"

In [ ]:
# Lecture des données test
test_id = gt_id[idx_test]
f'échantillons test: {test_id.shape[0]} pixels, {len(np.unique(test_id))} objets'

'échantillons test: 207485 pixels, 800 objets'

# **Exemple d'entraînement avec un classifieur Random forest**

### **Création d'un jeu de validation**

Il vous est recommandé de créer un **jeu de validation** à partir des échantillons d'entraînement afin de pouvoir jauger les performances de vos modèles. Notez qu'il est préférable de séparer le jeu d'entraînement en gardant la distribution des classes. De surcroît, afin d'éviter l'**autocorrélation spatiale** (contiguïté des échantillons), il est préférable que tous les pixels ayant le même identifiant se retrouvent soit dans le nouveau jeu d'entraînement ou le jeu de validation.

In [ ]:
# Récupérer les positions des nouveaux échantillons d'entraînement et validation
idx_train = np.where ( np.isin(gt_id,train_id) )
idx_valid = np.where ( np.isin(gt_id,valid_id) )

In [ ]:
# Lecture des nouvelles données d'entraînement et de validation
train_X = np.column_stack ( ( blue_ts_norm[idx_train], green_ts_norm[idx_train], red_ts_norm[idx_train], nir_ts_norm[idx_train] ) )
train_y = gt_class[idx_train]

valid_X = np.column_stack ( ( blue_ts_norm[idx_valid], green_ts_norm[idx_valid], red_ts_norm[idx_valid], nir_ts_norm[idx_valid] ) )
valid_y = gt_class[idx_valid]

train_X.shape, train_y.shape, valid_X.shape, valid_y.shape

((0, 32), (0,), (0, 32), (0,))

# **Explorer des modèles de deep learning**

Vous pourrez tester des **MLP** (Multi-layer perceptron), des **CNN** (Réseaux de neurones convolutionnels) adaptés signal 1D (série temporelle) ou 2D (image), des **RNN** (Réseaux de neurones récurrents) et même des combinaisons de ces modèles. Soyez créatifs.

In [ ]:
# create pickles of variables needed to write patches to memory
%ls Data/Sean_pickles

blue_ts_norm.pkl   gt_class.pkl     red_ts_norm.pkl
green_ts_norm.pkl  nir_ts_norm.pkl


In [ ]:
# save in my folder in Google Drive, the classes needed to generate 9x9 images from samples
# I can't just run this code in Colab as memory requirements are too high for Colab's limits
pickle.dump(gt_class, open(f'Data/Sean_pickles/gt_class.pkl',"wb"))
pickle.dump(blue_ts_norm, open(f'Data/Sean_pickles/blue_ts_norm.pkl',"wb"))
pickle.dump(green_ts_norm, open(f'Data/Sean_pickles/green_ts_norm.pkl',"wb"))
pickle.dump(red_ts_norm, open(f'Data/Sean_pickles/red_ts_norm.pkl',"wb"))
pickle.dump(nir_ts_norm, open(f'Data/Sean_pickles/nir_ts_norm.pkl',"wb"))